### Launch EVA DB
Run the command `python eva.py` in the server where you want to deploy EVA

### Establish Connection With Eva DB

In [1]:
import sys
sys.path.insert(0,'..')
from src.server.db_api import connect

In [2]:
# import nest_asyncio
# nest_asyncio.apply()
connection = connect(host = '0.0.0.0', port = 5432) # hostname, port of the server where EVADB is running

In [3]:
cursor = connection.cursor()

### Upload the survellience video to analyse

In [4]:
cursor.execute('UPLOAD INFILE "../data/ua_detrac/ua_detrac.mp4" PATH "video.mp4";')
response = cursor.fetch_one()

### Visualize Video

In [5]:
from ipywidgets import Video
Video.from_file("../data/ua_detrac/ua_detrac.mp4", embed=True)

Video(value=b'\x00\x00\x00 ftypisom\x00\x00\x02\x00isomiso2avc1mp41\x00\x00\x00\x08free\x00\x19K[mdat\x00\x00\…

### Load video into EVA

In [6]:
cursor.execute('LOAD DATA INFILE "test_video.mp4" INTO MyVideo;')
response = cursor.fetch_one()

### Optional - Register FasterRCNN (object detection) model into EVA 

In [7]:
cursor.execute("""CREATE UDF IF NOT EXISTS FastRCNNObjectDetector 
                  INPUT  (frame NDARRAY UINT8(3, 256, 256))
                  OUTPUT (labels NDARRAY STR(20), bboxes NDARRAY INT32(20, 4))
                  TYPE  Classification
                  IMPL  'src/udfs/fastrcnn_object_detector.py'""")
response = cursor.fetch_one()

### Run Object detector on the video

In [8]:
cursor.execute("""SELECT id, Unnest(FastRCNNObjectDetector(data)) FROM MyVideo""")
response = cursor.fetch_one()

### Visualize output of Object detector on the video

In [9]:
import cv2
def annotate_video(input_path, output_path):
    color=(0,255,0)
    thickness=3

    vcap = cv2.VideoCapture(input_path)
    width = int(vcap.get(3))
    height = int(vcap.get(4))
    fps = vcap.get(5)
    fourcc = cv2.VideoWriter_fourcc(*'H264') #codec
    video=cv2.VideoWriter(output_path, fourcc, fps, (width,height))

    frame_id = 0
    # Capture frame-by-frame
    ret, frame = vcap.read()  # ret = 1 if the video is captured; frame is the image

    while ret:
        df = response.batch.frames
        df = df[['boxes', 'labels']][df.id == frame_id]
        if df.size:
            for bbox, label in df.values:
                x1, y1 = bbox[0]
                x2, y2 = bbox[1]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                img=cv2.rectangle(frame, (x1, y1), (x2, y2), color, thickness) # object bbox
                cv2.putText(img, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, thickness-1) # object label
            video.write(img)

        frame_id+=1
        ret, frame = vcap.read()

    video.release()
    vcap.release()

In [10]:
from ipywidgets import Video
input_path = '../data/ua_detrac/ua_detrac.mp4'
output_path = 'video.mp4'
annotate_video(input_path, output_path)
Video.from_file(output_path)

Video(value=b'\x00\x00\x00\x1cftypisom\x00\x00\x02\x00isomiso2mp41\x00\x00\x00\x08free\x00\xdai\\mdat\x82I\x83…